<a href="https://colab.research.google.com/github/twumasimb/Image-Restoration-Project-CSCE-5218-/blob/main/image_restoration_thru_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Getting all the models we will be testing
# """
# Old-photos to Life
# """
# !git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git

# """
# GFPGAN
# """
# !git clone https://github.com/TencentARC/GFPGAN.git

# """
# CycleISP
# """
# !git clone https://github.com/swz30/CycleISP.git

Bringing Old Images to Life Using VAEs

In [ ]:
# %cd /content/Bringing-Old-Photos-Back-to-Life/

# %cd Face_Enhancement/models/networks/
# !git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
# !cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
# %cd ../../../

# %cd Global/detection_models
# !git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
# !cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
# %cd ../../

# %cd Face_Detection/
# !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
# !bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
# %cd ../

# %cd Face_Enhancement/
# !wget https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life/releases/download/v1.0/face_checkpoints.zip
# !unzip face_checkpoints.zip
# %cd ../

# %cd Global/
# !wget https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life/releases/download/v1.0/global_checkpoints.zip
# !unzip global_checkpoints.zip
# %cd ../

# !pip install -r requirements.txt

In [ ]:
# %cd /content/

# %cd GFPGAN/
# # We use BasicSR for both training and inference
# !pip install basicsr
# # We use face detection and face restoration helper in the facexlib package
# !pip install facexlib
# # Install other depencencies
# !pip install -r requirements.txt
# !python setup.py develop
# !pip install realesrgan  # used for enhancing the background (non-face) regions
# # Now we use the V1.3 model for the demo
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
# !cp -r /content/drive/MyDrive/Notebooks/raw/ /content/datasets/inputs

**EXPERIMENT 1**

In This Phase, the Model goes as follows ISP ---> GFPGAN & ISP ---> VAE

**ISP ---> VAE**

In [ ]:
# Dealing with clean data from ISP
import os

%cd /content/datasets/
input_folder = "inputs/clean"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp1/
output_folder = "vae-clean"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/Bringing-Old-Photos-Back-to-Life
!python run.py --input_folder /content/datasets/inputs/clean/ --output_folder /content/datasets/outputs/exp1/vae-clean/ --GPU -1

# Dealing with Raw Data

%cd /content/datasets/
input_folder = "inputs/raw"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp1/
output_folder = "vae-raw"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/Bringing-Old-Photos-Back-to-Life
!python run.py --input_folder /content/datasets/inputs/raw/ --output_folder /content/datasets/outputs/exp1/vae-raw/ --GPU -1

**ISP/RAW ---> GFPGAN**

In [ ]:
import os

# Dealing with clean data from ISP

%cd /content/datasets/
input_folder = "inputs/clean"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp1/
output_folder = "gfpgan-clean"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/GFPGAN/
!python inference_gfpgan.py -i  /content/datasets/inputs/clean/ -o /content/datasets/outputs/exp1/gfpgan-clean/ -v 1.3 -s 2 --bg_upsampler realesrgan

# Dealing with Raw Data

%cd /content/datasets/
input_folder = "inputs/raw"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp1/
output_folder = "gfpgan-raw"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/GFPGAN/
!python inference_gfpgan.py -i  /content/datasets/inputs/raw/ -o /content/datasets/outputs/exp1/gfpgan-raw/ -v 1.3 -s 2 --bg_upsampler realesrgan


**Experiment 2**

We are feeding the ouputs of each of the models into the other model 

**Raw-GFPGAN/ISP-GFPGAN --> VAE**

In [ ]:
import os

# Dealing with clean data from ISP

%cd /content/datasets/outputs/
input_folder = "exp1/gfpgan-clean/restored_imgs"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp2/
output_folder = "vae-clean"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/Bringing-Old-Photos-Back-to-Life
!python run.py --input_folder /content/datasets/outputs/exp1/gfpgan-clean/restored_imgs/ --output_folder /content/datasets/outputs/exp2/vae-clean/ --GPU -1

# Dealing with Raw Data

%cd /content/datasets/outputs/
input_folder = "exp1/gfpgan-raw/restored_imgs"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp2/
output_folder = "vae-raw"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/Bringing-Old-Photos-Back-to-Life
!python run.py --input_folder /content/datasets/outputs/exp1/gfpgan-raw/restored_imgs/ --output_folder /content/datasets/outputs/exp2/vae-raw/ --GPU -1

**Raw-VAE/ISP-VAE --> GFPGAN**

In [ ]:
import os

# Dealing with clean data from ISP

%cd /content/datasets/outputs/
input_folder = "exp1/vae-clean/final_output"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp2/
output_folder = "gfpgan-clean"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/GFPGAN/
!python inference_gfpgan.py -i  /content/datasets/outputs/exp1/vae-clean/final_output/ -o /content/datasets/outputs/exp2/gfpgan-clean/ -v 1.3 -s 2 --bg_upsampler realesrgan

# Dealing with Raw Data

%cd /content/datasets/outputs/
input_folder = "exp1/vae-raw/final_output"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)

%cd /content/datasets/outputs/exp2/
output_folder = "gfpgan-raw"
basepath = os.getcwd()
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/GFPGAN/
!python inference_gfpgan.py -i  /content/datasets/outputs/exp1/vae-raw/final_output/ -o /content/datasets/outputs/exp2/gfpgan-raw/ -v 1.3 -s 2 --bg_upsampler realesrgan


In [15]:
!cp -r /content/datasets /content/drive/MyDrive/Notebooks

In [ ]:
%cd /content/
# import eval_pred

import math
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as transforms
from torchmetrics import PeakSignalNoiseRatio
import torch.nn.functional as F

def image2tensor(image):
  """
  This function takes the path to an image and convert it into a tensor
  """
  image = Image.open(image)
  transform = transforms.Compose([
      transforms.PILToTensor()
  ])
  img_tensor = transform(image)
  return img_tensor

def psnr(generated, input_img):
  psnr = PeakSignalNoiseRatio()
  preds = torch.tensor(image2tensor(generated))
  target = torch.tensor(image2tensor(input_img))
  if(target.size() != preds.size()):
    preds = F.interpolate(preds, size=target.size())
  psnr_val = psnr(preds, target)
  return psnr_val.to_numpy()

#Evaluate the PSNR of generated image
raw_img = '/content/datasets/noisy/celeba-no-noise/Heng_Fan.png'
gen_img = '/content/datasets/clean/vae/final_output/Heng_Fan.png'
gfpgan_eval = psnr(raw_img, gen_img)

In [ ]:
# SSIM
import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')